## Step 1: Data loading and preprocessing

In [1]:
# Essentials
import pandas as pd
import numpy as np

# Library required to change date format
from datetime import datetime

# Ignore all FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Loading different brokers dataframes
angel_one = pd.read_json('/Users/shounak/Documents/Cuvette/InterviewProj/AngelBrokingScripMaster.json')
fyers_capital_market = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/FyersCapitalMarket.csv')
fyers_equity_derivatives = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/fyersED.csv')

# Below datasframes would include more NaN values in unified dataframe hence not used
#fyers_BSE_capital_market = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/Fyers_BSE_CM.csv')
#fyers_MCX_commodity = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/Fyers_MCX_COM.csv')
#fyers_nse_currency_derivatives = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/Fyers_NSE_CD.csv')

kotakcash = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/kotakcash.txt', delimiter='|')
kotak_fao = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/Kotakf&o.txt', delimiter='|')
zerodha = pd.read_csv('/Users/shounak/Documents/Cuvette/InterviewProj/zerodha.csv')

In [3]:
# Function to convert date format for angel_one 
def convert_date(date_str):
    if date_str:  # Check if the string is not empty
        date_obj = datetime.strptime(date_str, '%d%b%Y')
        return date_obj.strftime('%Y-%m-%d')
    return None  # Return None for empty strings

# Apply the conversion function to the 'expiry' column
angel_one['expiry'] = angel_one['expiry'].apply(convert_date)

#### Forming single dataframe of fyers

In [4]:
# List of DataFrames to concatenate 
#dataframes = [fyers_capital_market, fyers_equity_derivatives, fyers_BSE_capital_market, fyers_MCX_commodity, fyers_nse_currency_derivatives]
dataframes = [fyers_capital_market, fyers_equity_derivatives]

# Standardize columns and drop unnecessary ones for each DataFrame
for df in dataframes:
    df.columns = [
        "instrument_id", "name", "lot_size", "tick_size", "min_order_qty", "isin", "trading_hours",
        "expiry_date", "placeholder1", "exchange_symbol", "other_info", "close_price", "status",
        "symbol", "price_factor", "price_multiplier", "market_segment", "unique_code", 
        "placeholder2", "turnover", "open_interest"
    ]
    df.drop(columns=['placeholder1', 'placeholder2', 'turnover', 'open_interest', 'status', 
                     'min_order_qty', 'trading_hours', 'close_price', 'price_factor', 'isin', 
                     'other_info'], inplace=True)
    print(df.columns)

# Concatenate all DataFrames vertically
fyers = pd.concat(dataframes, axis=0, ignore_index=True)

#Displaying fyers as a complete dataframe
fyers.head()

Index(['instrument_id', 'name', 'lot_size', 'tick_size', 'expiry_date',
       'exchange_symbol', 'symbol', 'price_multiplier', 'market_segment',
       'unique_code'],
      dtype='object')
Index(['instrument_id', 'name', 'lot_size', 'tick_size', 'expiry_date',
       'exchange_symbol', 'symbol', 'price_multiplier', 'market_segment',
       'unique_code'],
      dtype='object')


,instrument_id,name,lot_size,tick_size,expiry_date,exchange_symbol,symbol,price_multiplier,market_segment,unique_code
0,101000000010,ABAN OFFSHORE LTD.,0,1,2024-10-24,NSE:ABAN-EQ,ABAN,-1.0,XX,101000000010
1,1010000000100,AMARA RAJA ENERGY MOB LTD,0,1,2024-10-24,NSE:ARE&M-EQ,ARE&M,-1.0,XX,1010000000100
2,10100000001000,SDL MH 6.56% 2032,5,100,2024-10-24,NSE:656MH32-SG,656MH32,-1.0,XX,10100000001000
3,10100000001001,SEC RED NCD 9.40% SR. V,2,1,2024-10-24,NSE:94SFL28-YL,94SFL28,-1.0,XX,10100000001001
4,10100000001004,SDL AP 6.79% 2034,5,100,2024-10-24,NSE:679AP34-SG,679AP34,-1.0,XX,10100000001004


#### Forming single dataframe of kotak

In [5]:
# Creating list of different dataframes
kotak_df = [kotakcash, kotak_fao]

# Checking the column names for each dataframe
for df in kotak_df:
    print(df.columns)

Index(['instrumentToken', 'instrumentName', 'name', 'lastPrice', 'expiry',
       'strike', 'tickSize', 'lotSize', 'instrumentType', 'segment',
       'exchange', 'isin', 'multiplier', 'exchangeToken', 'OptionType'],
      dtype='object')
Index(['instrumentToken', 'instrumentName', 'name', 'lastPrice', 'expiry',
       'strike', 'tickSize', 'lotSize', 'instrumentType', 'segment',
       'exchange', 'isin', 'multiplier', 'exchangeToken', 'optionType'],
      dtype='object')


In [6]:
# Standardising column names
kotak_fao.columns=['instrumentToken', 'instrumentName', 'name', 'lastPrice', 'expiry',
       'strike', 'tickSize', 'lotSize', 'instrumentType', 'segment',
       'exchange', 'isin', 'multiplier', 'exchangeToken', 'OptionType']

# Dropping not so relevant columns
for df in kotak_df:
    df.drop(columns = ['name', 'OptionType', 'lastPrice', 'isin'], inplace=True )
    
# Concatenate all DataFrames vertically
kotak = pd.concat(kotak_df, axis=0, ignore_index=True)

kotak.head()

,instrumentToken,instrumentName,expiry,strike,tickSize,lotSize,instrumentType,segment,exchange,multiplier,exchangeToken
0,9516,20MICRONS,0,0.0,0.05,1,EQ,CASH,NSE,1,16921
1,43865,3IINFOLTD,0,0.0,0.05,1,EQ,CASH,NSE,1,6232
2,891,3MINDIA,0,0.0,0.05,1,EQ,CASH,NSE,1,474
3,607,3PLAND,0,0.0,0.05,1,EQ,CASH,NSE,1,2595
4,13140,5PAISA,0,0.0,0.05,1,EQ,CASH,NSE,1,445


#### Now there's single data frame for angel_one, fyers, kotak and zerodha

In [7]:
# List of dataframes
data_files = [angel_one, fyers, kotak, zerodha]

# Function to clean each dataframe
def clean_dataframe(df):
    # Replace empty strings in all columns with NaN
    df.replace('', np.nan, inplace=True)
    
    # Remove duplicate rows
    df.drop_duplicates(inplace=True)
    
    # Drop columns with more than 50% missing values
    threshold = len(df) * 0.5
    df.dropna(thresh=threshold, axis=1, inplace=True)
    
    # Fill NaN values for categorical columns with the mode
    catg_cols = df.select_dtypes(include=['object']).columns
    df[catg_cols] = df[catg_cols].fillna(df[catg_cols].mode().iloc[0])
    
    # Fill NaN values for numeric columns with the mode
    num_cols = df.select_dtypes(include=['number']).columns
    df[num_cols] = df[num_cols].fillna(df[num_cols].mode().iloc[0])

# Apply the function to each dataframe in the list
for df in data_files:
    clean_dataframe(df)

In [8]:
# Checking column names for each dataframes
print("angel_one columns:", angel_one.columns, end ='\n\n')
print("fyers columns:", fyers_capital_market.columns, end ='\n\n')
print("kotak columns:", kotak_fao.columns, end ='\n\n')
print("zerodha columns:", zerodha.columns, end ='\n\n')

angel_one columns: Index(['token', 'symbol', 'name', 'expiry', 'strike', 'lotsize',
       'instrumenttype', 'exch_seg', 'tick_size'],
      dtype='object')

fyers columns: Index(['instrument_id', 'name', 'lot_size', 'tick_size', 'expiry_date',
       'exchange_symbol', 'symbol', 'price_multiplier', 'market_segment',
       'unique_code'],
      dtype='object')

kotak columns: Index(['instrumentToken', 'instrumentName', 'expiry', 'strike', 'tickSize',
       'lotSize', 'instrumentType', 'segment', 'exchange', 'multiplier',
       'exchangeToken'],
      dtype='object')

zerodha columns: Index(['instrument_token', 'exchange_token', 'tradingsymbol', 'name',
       'last_price', 'expiry', 'strike', 'tick_size', 'lot_size',
       'instrument_type', 'segment', 'exchange'],
      dtype='object')



## Step 2: Dataframe Mapping

In [9]:
# Standard column names mapping
column_mappings = {
    'token': 'instrument_id',
    'instrumentToken': 'instrument_id',
    'instrument_token': 'instrument_id',
    'expiry': 'expiry_date', 
    'exchange_token': 'exchange_token',
    'symbol': 'tradingsymbol',
    'instrumentName': 'name',
    'lastPrice': 'last_price',
    'last_price': 'last_price',
    'strike': 'strike_price',
    'lotsize': 'lot_size',
    'lotSize': 'lot_size',
    'tickSize': 'tick_size',
    'instrumenttype': 'instrument_type',
    'instrumentType': 'instrument_type',
    'exch_seg': 'exchange',
    'segment': 'market_segment'
}

# Rename columns across all dataframes in data_files
for df in data_files:
    df.rename(columns=column_mappings, inplace=True)

# Checking column names for each dataframe to verify renaming
print("angel_one columns:", angel_one.columns, end='\n\n')
print("fyers columns:", fyers.columns, end='\n\n')
print("kotak columns:", kotak.columns, end='\n\n')
print("zerodha columns:", zerodha.columns, end='\n\n')


angel_one columns: Index(['instrument_id', 'tradingsymbol', 'name', 'expiry_date', 'strike_price',
       'lot_size', 'instrument_type', 'exchange', 'tick_size'],
      dtype='object')

fyers columns: Index(['instrument_id', 'name', 'lot_size', 'tick_size', 'expiry_date',
       'exchange_symbol', 'tradingsymbol', 'price_multiplier',
       'market_segment', 'unique_code'],
      dtype='object')

kotak columns: Index(['instrument_id', 'name', 'expiry_date', 'strike_price', 'tick_size',
       'lot_size', 'instrument_type', 'market_segment', 'exchange',
       'multiplier', 'exchangeToken'],
      dtype='object')

zerodha columns: Index(['instrument_id', 'exchange_token', 'tradingsymbol', 'name',
       'last_price', 'expiry_date', 'strike_price', 'tick_size', 'lot_size',
       'instrument_type', 'market_segment', 'exchange'],
      dtype='object')



In [10]:
# Getting a brief of contents in each dataframe
datafile_names = ['angel_one', 'fyers', 'kotak', 'zerodha']
i=0

for df in data_files:
    print(datafile_names[i]+': ')
    print(df.head())
    i+=1

angel_one: 
  instrument_id tradingsymbol       name expiry_date  strike_price  lot_size  \
0          6293    GULPOLY-BL    GULPOLY  2024-10-31            -1         1   
1         24051    739UK30-SG    739UK30  2024-10-31            -1       100   
2          3475    THERMAX-EQ    THERMAX  2024-10-31            -1         1   
3          3523     69MZ33-SG     69MZ33  2024-10-31            -1       100   
4         17996  PGIMCSR2G-MF  PGIMCSR2G  2024-10-31            -1         1   

  instrument_type exchange  tick_size  
0          OPTSTK      NSE          1  
1          OPTSTK      NSE          1  
2          OPTSTK      NSE          5  
3          OPTSTK      NSE          1  
4          OPTSTK      NSE          1  
fyers: 
    instrument_id                       name  lot_size  tick_size expiry_date  \
0    101000000010         ABAN OFFSHORE LTD.         0          1  2024-10-24   
1   1010000000100  AMARA RAJA ENERGY MOB LTD         0          1  2024-10-24   
2  1010000000100

In [11]:
# Convert instrument_id to string for its uniformity across all dataframes
for df in data_files:
    df['instrument_id'] = df['instrument_id'].astype(str)

In [12]:
# Renaming columns to add broker-specific prefix
angel_one = angel_one.add_prefix('angel_one_')
fyers = fyers.add_prefix('fyers_')
kotak = kotak.add_prefix('kotak_')
zerodha = zerodha.add_prefix('zerodha_')

In [13]:
# Merge based on the specified keys (e.g., tradingsymbol, instrument_id, name), using 'outer' join for a complete mapping
mdf = angel_one.merge(
    fyers, left_on=['angel_one_tradingsymbol', 'angel_one_instrument_id'],
    right_on=['fyers_tradingsymbol', 'fyers_instrument_id'],
    how='outer'
).merge(
    kotak, left_on=['angel_one_name','angel_one_instrument_id'],
    right_on=['kotak_name','kotak_instrument_id'],
    how='outer'
).merge(
    zerodha, left_on=['angel_one_tradingsymbol', 'angel_one_instrument_id'],
    right_on=['zerodha_tradingsymbol', 'zerodha_instrument_id'],
    how='outer'
)

#Dropping duplicate columns created due to merge keys
mdf = mdf.loc[:, ~mdf.columns.duplicated()]

In [14]:
# Display the unified dataframes
mdf.head()

,angel_one_instrument_id,angel_one_tradingsymbol,angel_one_name,angel_one_expiry_date,angel_one_strike_price,angel_one_lot_size,angel_one_instrument_type,angel_one_exchange,angel_one_tick_size,fyers_instrument_id,...,zerodha_tradingsymbol,zerodha_name,zerodha_last_price,zerodha_expiry_date,zerodha_strike_price,zerodha_tick_size,zerodha_lot_size,zerodha_instrument_type,zerodha_market_segment,zerodha_exchange
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,001HCCL26,USDINR,0.0,2024-10-31,0.0,0.01,1.0,EQ,BSE,BSE
1,974248,001HCCL26,001HCCL26,2024-10-31,-1.0,1.0,OPTSTK,BSE,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,001HCCL29,USDINR,0.0,2024-10-31,0.0,0.01,1.0,EQ,BSE,BSE
3,974246,001HCCL29,001HCCL29,2024-10-31,-1.0,1.0,OPTSTK,BSE,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,001HCCL29A,USDINR,0.0,2024-10-31,0.0,0.01,1.0,EQ,BSE,BSE


In [15]:
# Drop rows with fewer than 10 non-NaN values
#mdf = mdf.dropna(thresh=10)

In [16]:
# replace NaN Values with Not Available
mdf.fillna("Not Available", inplace=True)
mdf

,angel_one_instrument_id,angel_one_tradingsymbol,angel_one_name,angel_one_expiry_date,angel_one_strike_price,angel_one_lot_size,angel_one_instrument_type,angel_one_exchange,angel_one_tick_size,fyers_instrument_id,...,zerodha_tradingsymbol,zerodha_name,zerodha_last_price,zerodha_expiry_date,zerodha_strike_price,zerodha_tick_size,zerodha_lot_size,zerodha_instrument_type,zerodha_market_segment,zerodha_exchange
0,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,...,001HCCL26,USDINR,0.0,2024-10-31,0.0,0.01,1.0,EQ,BSE,BSE
1,974248,001HCCL26,001HCCL26,2024-10-31,-1.0,1.0,OPTSTK,BSE,1.0,Not Available,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
2,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,...,001HCCL29,USDINR,0.0,2024-10-31,0.0,0.01,1.0,EQ,BSE,BSE
3,974246,001HCCL29,001HCCL29,2024-10-31,-1.0,1.0,OPTSTK,BSE,1.0,Not Available,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
4,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,...,001HCCL29A,USDINR,0.0,2024-10-31,0.0,0.01,1.0,EQ,BSE,BSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409380,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,101124122656416,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
409381,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,101124122656417,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
409382,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,101124122663894,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available
409383,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,101124122663895,...,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available,Not Available


In [17]:
# Export the merged DataFrame to a CSV file with the new name
mdf.to_csv('/Users/shounak/Documents/Cuvette/InterviewProj/Unified_script_master.csv', index=False)